<a href="https://colab.research.google.com/github/mhanauer/pg_report/blob/main/pg_report.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!git clone https://github.com/mhanauer/pg_report.git

Cloning into 'pg_report'...
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 7 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (7/7), done.


In [3]:
!pip install xgboost
!pip install lightgbm
!pip install scikit-optimize
import pandas as pd
import sklearn
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from skopt import BayesSearchCV
import xgboost as xgb
import lightgbm as lgb
from sklearn.metrics import mean_absolute_error,r2_score,mean_squared_error
import numpy as np
from joblib import dump, load
from sklearn.model_selection import RepeatedKFold
%cd pg_report/

     |████████████████████████████████| 100 kB 3.2 MB/s 
/content/pg_report


Step: Load in data
(Please present this answer visually)
How does our conversion rate change through the funnel overall and by marketing channels?

Create overall conversation rate which is the percentage of sign_and_exam to started navigator.

Then analyze this conversation rate by marketing change.




In [8]:
funnel_path = pd.read_csv("funnel_path.csv")
users = pd.read_csv("users.csv")
funnel_path_users = funnel_path.merge(users, on='user_id', how='left')
funnel_path_users


,user_id,funnel_steps,age,device,income,num_conditions,premium,marketing_channels
0,1001,collect_contact_info,56,desktop,157490,5,2490.123774,instagram
1,1001,started_navigator,56,desktop,157490,5,2490.123774,instagram
2,1002,application_submit,34,desktop,138171,3,1906.228442,paid_search_nb
3,1002,collect_contact_info,34,desktop,138171,3,1906.228442,paid_search_nb
4,1002,phone_connect,34,desktop,138171,3,1906.228442,paid_search_nb
...,...,...,...,...,...,...,...,...
27624,10999,collect_contact_info,20,mobile_ios,163641,4,1786.076634,podcast
27625,10999,started_navigator,20,mobile_ios,163641,4,1786.076634,podcast
27626,10999,viewed_quotes,20,mobile_ios,163641,4,1786.076634,podcast
27627,11000,collect_contact_info,21,mobile_ios,169217,3,1756.614104,podcast


Calculate conversation rate overall and by channel

In [10]:
test = pd.DataFrame(funnel_path_users[["funnel_steps"]].value_counts())
test

,0
funnel_steps,
started_navigator,10000
collect_contact_info,7220
viewed_quotes,5245
application_submit,2855
phone_connect,1525
sign_and_exam,784
